In [ ]:
### Import Dependencies

In [1]:
import pandas as pd
import statsapi
from pprint import pprint
import requests
from bs4 import BeautifulSoup
import json
from flask import jsonify

In [2]:
#Define url to pull WAR leaders
espn_url = "https://www.espn.com/mlb/war/leaders/_/type/seasonal/year/20"

In [4]:
#define list of stat years to pull and create empty list of DF's to store results
year_list = ['19', '18', '17', '16', '15']
df_list = []

In [5]:
#scrape stat leaders
for year in year_list:
    query_url = espn_url+year
    qdfl = pd.read_html(query_url)
    qdfl[0]['season'] = year
    df_list.append(qdfl[0])

In [6]:
#concat dfs into one
data = pd.concat(df_list)
data.head()

,0,1,2,3,4,5,6,7,8,9,season
0,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,19
1,RK,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,1,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,2,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,3,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [7]:
cols = ['rank', 'player', 'WAR', 'OFF', 'DEF', 'WAA', 'TRPG', 'ORPG', 'RAA', 'WAAWP', 'Season']

In [8]:
#rename columns and drop rank column
data.columns = cols
del data['rank']
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
0,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,2019 Season WAR Leaders,19
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19


In [9]:
#drop NAs
data = data.dropna(subset=['player'])

In [10]:
#drop no data holding columns
data = data[~data.player.str.contains("WAR")]

In [11]:
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season
1,PLAYER,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,19
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19


In [12]:
# new data frame with split value columns 
new = data["player"].str.split(" ", n = 1, expand = True) 
  
# making separate first name column from new data frame 
data["First Name"]= new[0] 
data["Last Name"]= new[1] 
  
#reset index
data = data.drop(data.index[0])
data.reset_index()
data.head()

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name
2,Cody Bellinger,9,6.6,1.9,7,5,4.7,72.9,0.5,19,Cody,Bellinger
3,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5,62,0.5,19,Alex,Bregman
4,Mike Trout,8.3,8.3,0.3,6.3,5.5,5,65.7,0.5,19,Mike,Trout
5,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5,58.5,0.5,19,Marcus,Semien
6,Justin Verlander,7.8,0,0,5.8,5,3.4,56.6,0.7,19,Justin,Verlander


In [13]:
#drop duplicate entries
data.drop_duplicates(subset ="player", keep ='first', inplace = True)

In [14]:

#read in master list, change encoding to make it readable
id_list = pd.read_csv('master_id_list.csv', encoding='latin-1')
id_df = pd.DataFrame(id_list)[['mlb_name','espn_id','espn_name', 'espn_pos']].dropna()


In [15]:
#merge ids and player data
new_data = pd.merge(data, id_df,  how='left', left_on=['player'], right_on = ['mlb_name'])

In [16]:
#do not overwrite this csv!!!
#new_data.to_csv('player_list.csv')

In [11]:
clean_data = pd.read_csv('../player_list.csv')
clean_data = clean_data.drop(columns=['Unnamed: 0'])
clean_data

,player,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name,mlb_name,espn_id,espn_name,espn_pos
0,Cody Bellinger,9.0,6.6,1.9,7.0,5.0,4.7,72.9,0.5,19,Cody,Bellinger,Cody Bellinger,33912,Cody Bellinger,CF
1,Alex Bregman,8.4,7.7,1.2,6.1,5.3,5.0,62.0,0.5,19,Alex,Bregman,Alex Bregman,34886,Alex Bregman,3B
2,Mike Trout,8.3,8.3,0.3,6.3,5.5,5.0,65.7,0.5,19,Mike,Trout,Mike Trout,30836,Mike Trout,CF
3,Marcus Semien,8.1,7.5,1.5,5.8,5.3,5.0,58.5,0.5,19,Marcus,Semien,Marcus Semien,32146,Marcus Semien,SS
4,Justin Verlander,7.8,0.0,0.0,5.8,5.0,3.4,56.6,0.7,19,Justin,Verlander,Justin Verlander,6341,Justin Verlander,SP
5,Mike Minor,7.8,0.0,0.0,5.9,5.0,3.2,57.0,0.7,19,Mike,Minor,Mike Minor,30624,Mike Minor,SP
6,Lance Lynn,7.6,0.0,0.0,5.7,5.0,3.3,55.6,0.7,19,Lance,Lynn,Lance Lynn,30820,Lance Lynn,SP
7,Jacob deGrom,7.3,0.0,0.0,5.6,4.7,3.1,50.2,0.7,19,Jacob,deGrom,Jacob deGrom,32796,Jacob deGrom,SP
8,Christian Yelich,7.1,7.3,-0.7,5.4,5.2,4.7,57.7,0.5,19,Christian,Yelich,Christian Yelich,31283,Christian Yelich,RF
9,Ketel Marte,6.9,6.6,0.9,5.0,5.1,4.7,53.9,0.5,19,Ketel,Marte,Ketel Marte,32512,Ketel Marte,2B


In [13]:
#create empyty list for scraping query urls
q_urls=[]

In [14]:
#build query urls
for index, row in clean_data.iterrows():
    espn_tag = row['First Name']+ '-' +row['Last Name']
    espn_tag = espn_tag.lower()
    turl = 'http://www.espn.com/mlb/player/stats/_/id/' + str(row['espn_id']) + '/'+ espn_tag
    q_urls.append(turl)

In [15]:
#add to DF
clean_data['qurl'] = q_urls

In [16]:
#set index to player
clean_data = clean_data.set_index('player')

In [17]:
clean_data.head(3)

,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name,mlb_name,espn_id,espn_name,espn_pos,qurl
player,,,,,,,,,,,,,,,,
Cody Bellinger,9.0,6.6,1.9,7.0,5.0,4.7,72.9,0.5,19,Cody,Bellinger,Cody Bellinger,33912,Cody Bellinger,CF,http://www.espn.com/mlb/player/stats/_/id/3391...
Alex Bregman,8.4,7.7,1.2,6.1,5.3,5.0,62.0,0.5,19,Alex,Bregman,Alex Bregman,34886,Alex Bregman,3B,http://www.espn.com/mlb/player/stats/_/id/3488...
Mike Trout,8.3,8.3,0.3,6.3,5.5,5.0,65.7,0.5,19,Mike,Trout,Mike Trout,30836,Mike Trout,CF,http://www.espn.com/mlb/player/stats/_/id/3083...


In [23]:
#drop old index column
clean_data = clean_data.drop(columns=['Unnamed: 0'])

In [24]:
clean_data.head(3)

,WAR,OFF,DEF,WAA,TRPG,ORPG,RAA,WAAWP,Season,First Name,Last Name,mlb_name,espn_id,espn_name,espn_pos,qurl
player,,,,,,,,,,,,,,,,
Cody Bellinger,9.0,6.6,1.9,7.0,5.0,4.7,72.9,0.5,19,Cody,Bellinger,Cody Bellinger,33912,Cody Bellinger,CF,http://www.espn.com/mlb/player/stats/_/id/3391...
Alex Bregman,8.4,7.7,1.2,6.1,5.3,5.0,62.0,0.5,19,Alex,Bregman,Alex Bregman,34886,Alex Bregman,3B,http://www.espn.com/mlb/player/stats/_/id/3488...
Mike Trout,8.3,8.3,0.3,6.3,5.5,5.0,65.7,0.5,19,Mike,Trout,Mike Trout,30836,Mike Trout,CF,http://www.espn.com/mlb/player/stats/_/id/3083...


In [19]:
#scrape stats from ESPN, clean stats DF, and convert to a Mongo-ready list of dictionaries
mongo_list = []
for i in clean_data.index:
    temp_df = pd.read_html(clean_data['qurl'][i])
    #above returns a list of dfs, this line pulls the df that contains the stat table
    stat_df = temp_df[1]
    #dropping the blank row at the top and the summary row at the bottom
    stat_df.drop(stat_df.tail(1).index,inplace=True)
    stat_df.drop(stat_df.head(1).index,inplace=True)
    stat_df.columns = stat_df.iloc[0]
    stat_df = stat_df.drop(stat_df.index[0])
    #turning the df into a dictionary and adding to the mongo listS
    mongo_list.append(stat_df.to_dict())

In [50]:
mongo_list.json

['{"YEAR":{"2":"2017","3":"2018","4":"2019","5":"Total"},"TEAM":{"2":"LAD","3":"LAD","4":"LAD","5":"Total"},"GP":{"2":"132","3":"162","4":"156","5":"450"},"AB":{"2":"480","3":"557","4":"558","5":"1595"},"R":{"2":"87","3":"84","4":"121","5":"292"},"H":{"2":"128","3":"145","4":"170","5":"443"},"2B":{"2":"26","3":"28","4":"34","5":"88"},"3B":{"2":"4","3":"7","4":"3","5":"14"},"HR":{"2":"39","3":"25","4":"47","5":"111"},"RBI":{"2":"97","3":"76","4":"115","5":"288"},"BB":{"2":"64","3":"69","4":"95","5":"228"},"SO":{"2":"146","3":"151","4":"108","5":"405"},"SB":{"2":"10","3":"14","4":"15","5":"39"},"CS":{"2":"3","3":"1","4":"5","5":"9"},"AVG":{"2":".267","3":".260","4":".305","5":".278"},"OBP":{"2":".352","3":".343","4":".406","5":".368"},"SLG":{"2":".581","3":".470","4":".629","5":".559"},"OPS":{"2":".933","3":".813","4":"1.035","5":".927"},"WAR":{"2":"4.2","3":"4.2","4":"--","5":"--"}}',
 '{"YEAR":{"2":"2016","3":"2017","4":"2018","5":"2019","6":"Total"},"TEAM":{"2":"HOU","3":"HOU","4":"HO

In [51]:
#add name to dictionaries
for i in range(len(clean_data)):
    mongo_list[i]['name'] = clean_data['espn_name'][i]

TypeError: 'str' object does not support item assignment

In [28]:
#read in player photos list
%store -r player_photos

no stored variable player_photos


In [29]:
#add photo urls to dictionaries
for i in range(len(clean_data)):
    mongo_list[i]['photo_url'] = player_photos[i]

NameError: name 'player_photos' is not defined

In [30]:
#store mongo list of entires
%store mongo_list
    



Stored 'mongo_list' (list)


In [42]:
mongo_list

[{'YEAR': 2     2017
  3     2018
  4     2019
  5    Total
  Name: YEAR, dtype: object, 'TEAM': 2      LAD
  3      LAD
  4      LAD
  5    Total
  Name: TEAM, dtype: object, 'GP': 2    132
  3    162
  4    156
  5    450
  Name: GP, dtype: object, 'AB': 2     480
  3     557
  4     558
  5    1595
  Name: AB, dtype: object, 'R': 2     87
  3     84
  4    121
  5    292
  Name: R, dtype: object, 'H': 2    128
  3    145
  4    170
  5    443
  Name: H, dtype: object, '2B': 2    26
  3    28
  4    34
  5    88
  Name: 2B, dtype: object, '3B': 2     4
  3     7
  4     3
  5    14
  Name: 3B, dtype: object, 'HR': 2     39
  3     25
  4     47
  5    111
  Name: HR, dtype: object, 'RBI': 2     97
  3     76
  4    115
  5    288
  Name: RBI, dtype: object, 'BB': 2     64
  3     69
  4     95
  5    228
  Name: BB, dtype: object, 'SO': 2    146
  3    151
  4    108
  5    405
  Name: SO, dtype: object, 'SB': 2    10
  3    14
  4    15
  5    39
  Name: SB, dtype: object, 'CS': 2  

In [52]:
df = pd.DataFrame(mongo_list)

In [53]:
df2 = df.to_json(r'C:\Users\laurakeller\Desktop\baseballStats.json',orient = 'records')

In [12]:
json.dumps(df2)

NameError: name 'df2' is not defined